In [44]:
import itertools

opt = ['8', 'b', 't', '}', '7', '{', 'f', 'c', '6', 'a', '0']
opts = [ord(c) for c in opt]
perms5 = list(itertools.product(opts, repeat = 5))
print(len(perms5))
perms4 = list(itertools.product(opts, repeat = 4))
              
print(len(perms4))

161051
14641


In [45]:
from tqdm import tqdm

M = (1 << 127) - 1
d1 = dict()
p1 = 23
for perm in tqdm(perms5):
    pwL = [0, 2, 4, 6, 8]
    sm = sum([perm[i] * (p1 ** pwL[i]) for i in range(5)])
    d1[sm] = perm

100%|██████████| 161051/161051 [00:02<00:00, 59640.69it/s]


In [46]:
d2 = dict()
p2 = 23
for perm in tqdm(perms5):
    pwL = pwL = [1, 3, 5, 7, 9]
    sm = sum([perm[i] * (p2 ** pwL[i]) for i in range(5)])
    d2[sm] = perm

100%|██████████| 161051/161051 [00:02<00:00, 68063.15it/s]


In [62]:
import socket

def sendline(sock, data):
    sock.sendall(data + b"\n")

def recvline(sock):
    buf = b""
    while True:
        data = sock.recv(1)
        if not data:
            break
        buf += data
        if data == b"\n":
            break
    return buf

def recvuntil(sock, delim):
    buf = b""
    while True:
        data = sock.recv(1)
        if not data:
            break
        buf += data
        if delim in buf:
            break
    return buf




In [70]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('challs.actf.co', 32100))

recvuntil(sock, b":")
sendline(sock, b"1")
recvuntil(sock, b'>')
sendline(sock, b'23 170141183460469231731687303715884105704')
r1 = int(recvline(sock).strip())
r2 = int(recvline(sock).strip())
recvuntil(sock, b':')
sendline(sock, b'2')
recvuntil(sock, b'>')

RHS = [r1, r2]

p1_sm = (RHS[0] + RHS[1]) % M
p1_sm = (pow(2, -1, M) * p1_sm) % M
even_pow = None
for perm in tqdm(perms4):
    pwR = [10, 12, 14, 16]
    sm = sum([perm[i] * (p1 ** pwR[i]) for i in range(4)])
    need = (p1_sm - sm + M) % M
    if need in d1:
        even_pow = d1[need] + perm
        break
        
other_sm = (RHS[0] - RHS[1]) % M
other_sm = (pow(2, -1, M) * other_sm) % M
odd_pow = None
for perm in tqdm(perms4):
    pwR = [11, 13, 15, 17]
    sm = sum([perm[i] * (p2 ** pwR[i]) for i in range(4)])
    need = (other_sm - sm + M) % M
    if need in d2:
        odd_pow = d2[need] + perm
        break
        

coeffs = []
for i in range(18):
    if i % 2 == 0:
        coeffs.append(even_pow[i // 2])
    else:
        coeffs.append(odd_pow[i // 2])
        
print(coeffs)

to_send = " ".join([str(i) for i in coeffs])
sendline(sock, to_send.encode())
perms = recvline(sock).decode().strip()
perms = list(map(int, perms.split()))

print(perms)

flag = [0 for _ in range(len(coeffs))]
for idx, val in enumerate(perms):
    flag[val] = coeffs[idx]

flag = ''.join([chr(c) for c in flag])
print(flag)

 36%|███▌      | 5288/14641 [00:00<00:00, 22740.63it/s]


[98, 48, 97, 56, 102, 102, 116, 102, 56, 54, 55, 125, 123, 48, 97, 99, 55, 54]
[16, 10, 13, 11, 8, 3, 2, 5, 6, 9, 14, 17, 4, 7, 0, 1, 15, 12]
actf{f80f6086a77b}
